In [1]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import from_unixtime, col,to_date, date_format


In [2]:
spark = SparkSession.builder.appName('movie').getOrCreate()


25/03/13 10:04:58 WARN Utils: Your hostname, ali-ThinkBook-14-G2-ITL resolves to a loopback address: 127.0.1.1; using 172.16.8.114 instead (on interface wlp0s20f3)
25/03/13 10:04:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 10:04:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_data = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.data", 
                    sep="\t", 
                    schema="userId INT, movieId INT, rating FLOAT, timestamp LONG")

In [4]:
df_data.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|   196|    242|   3.0|881250949|
|   186|    302|   3.0|891717742|
|    22|    377|   1.0|878887116|
|   244|     51|   2.0|880606923|
|   166|    346|   1.0|886397596|
|   298|    474|   4.0|884182806|
|   115|    265|   2.0|881171488|
|   253|    465|   5.0|891628467|
|   305|    451|   3.0|886324817|
|     6|     86|   3.0|883603013|
|    62|    257|   2.0|879372434|
|   286|   1014|   5.0|879781125|
|   200|    222|   5.0|876042340|
|   210|     40|   3.0|891035994|
|   224|     29|   3.0|888104457|
|   303|    785|   3.0|879485318|
|   122|    387|   5.0|879270459|
|   194|    274|   2.0|879539794|
|   291|   1042|   4.0|874834944|
|   234|   1184|   2.0|892079237|
+------+-------+------+---------+
only showing top 20 rows



In [6]:
df_genre = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.genre", 
                           sep="|", 
                           schema="genre STRING, id INT")

df_info = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.info", 
                          sep=" ", 
                          schema="value INT, description STRING")

df_item = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.item", 
                          sep="|", 
                          schema="movieId INT, movieTitle STRING, releaseDate STRING, videoReleaseDate STRING, imdbUrl STRING, unknown INT, action INT, adventure INT, animation INT, childrens INT, comedy INT, crime INT, documentary INT, drama INT, fantasy INT, filmNoir INT, horror INT, musical INT, mystery INT, romance INT, sciFi INT, thriller INT, war INT, western INT")

df_occupation = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.occupation", 
                                 schema="occupation STRING")

df_user = spark.read.csv("file:///home/ali/Desktop/Movie Recommendation System/data/ml-100k/u.user", 
                          sep="|", 
                          schema="userId INT, age INT, gender STRING, occupation STRING, zipCode STRING")

In [7]:
df_genre.show()

+-----------+---+
|      genre| id|
+-----------+---+
|    unknown|  0|
|     Action|  1|
|  Adventure|  2|
|  Animation|  3|
| Children's|  4|
|     Comedy|  5|
|      Crime|  6|
|Documentary|  7|
|      Drama|  8|
|    Fantasy|  9|
|  Film-Noir| 10|
|     Horror| 11|
|    Musical| 12|
|    Mystery| 13|
|    Romance| 14|
|     Sci-Fi| 15|
|   Thriller| 16|
|        War| 17|
|    Western| 18|
+-----------+---+



In [8]:
df_item.show(truncate=False)

+-------+----------------------------------------------------+-----------+----------------+-------------------------------------------------------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
|movieId|movieTitle                                          |releaseDate|videoReleaseDate|imdbUrl                                                            |unknown|action|adventure|animation|childrens|comedy|crime|documentary|drama|fantasy|filmNoir|horror|musical|mystery|romance|sciFi|thriller|war|western|
+-------+----------------------------------------------------+-----------+----------------+-------------------------------------------------------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
|1      |Toy Story (1995)                                    |01-Ja

In [9]:
from pyspark.sql.functions import from_unixtime, col

df=df_data.join(df_user,'userId',how='inner').join(df_item,'movieId','inner').withColumn('datetime',from_unixtime(col("timestamp"))).drop('timestamp').withColumn('time',date_format(col("datetime"),"HH:mm:ss")).withColumn('date',to_date(col("datetime"))).drop('datetime')

In [10]:
df.show()

25/03/13 10:05:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------+------+---+------+-------------+-------+--------------------+-----------+----------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+--------+----------+
|movieId|userId|rating|age|gender|   occupation|zipCode|          movieTitle|releaseDate|videoReleaseDate|             imdbUrl|unknown|action|adventure|animation|childrens|comedy|crime|documentary|drama|fantasy|filmNoir|horror|musical|mystery|romance|sciFi|thriller|war|western|    time|      date|
+-------+------+------+---+------+-------------+-------+--------------------+-----------+----------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+--------+----------+
|    242|   196|   3.0| 49|     M|       writer|  55105|        Kolya (1996)|24-Jan-1997|            NU

In [11]:
df_data=df_data.drop('timestamp')

In [11]:
df_data.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   196|    242|   3.0|
|   186|    302|   3.0|
|    22|    377|   1.0|
|   244|     51|   2.0|
|   166|    346|   1.0|
|   298|    474|   4.0|
|   115|    265|   2.0|
|   253|    465|   5.0|
|   305|    451|   3.0|
|     6|     86|   3.0|
|    62|    257|   2.0|
|   286|   1014|   5.0|
|   200|    222|   5.0|
|   210|     40|   3.0|
|   224|     29|   3.0|
|   303|    785|   3.0|
|   122|    387|   5.0|
|   194|    274|   2.0|
|   291|   1042|   4.0|
|   234|   1184|   2.0|
+------+-------+------+
only showing top 20 rows



In [63]:
df_data

DataFrame[userId: int, movieId: int, rating: float]

In [12]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [13]:
df_data.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   196|    242|   3.0|
|   186|    302|   3.0|
|    22|    377|   1.0|
|   244|     51|   2.0|
|   166|    346|   1.0|
|   298|    474|   4.0|
|   115|    265|   2.0|
|   253|    465|   5.0|
|   305|    451|   3.0|
|     6|     86|   3.0|
|    62|    257|   2.0|
|   286|   1014|   5.0|
|   200|    222|   5.0|
|   210|     40|   3.0|
|   224|     29|   3.0|
|   303|    785|   3.0|
|   122|    387|   5.0|
|   194|    274|   2.0|
|   291|   1042|   4.0|
|   234|   1184|   2.0|
+------+-------+------+
only showing top 20 rows



In [14]:
df_data_tarin,df_data_test=df_data.randomSplit([0.8,0.2])

In [15]:
df_data_tarin.count()

79888

In [16]:
df_data_test.count()

20112

In [17]:
df_data.count()

100000

In [61]:
als=ALS(maxIter=10,rank=4,regParam=0.1,userCol='userId',itemCol='movieId',ratingCol='rating')

In [62]:
model=als.fit(df_data_tarin)

In [63]:
predictions=model.transform(df_data_test)

In [64]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|      1|   5.0| 3.8165877|
|     1|      7|   4.0|   4.03037|
|     1|     10|   3.0| 4.0765753|
|     1|     16|   5.0| 3.5653815|
|     1|     17|   3.0| 3.0659015|
|     1|     34|   2.0| 1.3376474|
|     1|     39|   4.0| 3.3385286|
|     1|     41|   2.0| 2.5630927|
|     1|     45|   5.0|  4.247238|
|     1|     51|   4.0| 3.1219783|
|     1|     56|   4.0|  4.376049|
|     1|     57|   5.0| 3.8636377|
|     1|     59|   5.0|  4.281887|
|     1|     66|   4.0| 2.8900228|
|     1|     68|   4.0| 3.3818574|
|     1|     75|   4.0| 3.8104255|
|     1|     76|   4.0| 3.4756641|
|     1|     81|   5.0|  3.902081|
|     1|     82|   5.0| 3.3808446|
|     1|     87|   5.0|  3.840701|
+------+-------+------+----------+
only showing top 20 rows



In [65]:
evaluator=RegressionEvaluator(metricName='rmse',predictionCol='prediction',labelCol='rating')

In [66]:
predictions=predictions.filter(col('prediction')!='nan')

In [67]:
rmse=evaluator.evaluate(predictions)

In [68]:
print(rmse)

0.9222796504357282


In [26]:
import datetime

In [27]:
print(datetime.datetime.now())

2025-03-12 22:15:29.411363


In [28]:
print(datetime.date.today())

2025-03-12


In [79]:
als=ALS(userCol='userId',itemCol='movieId',ratingCol='rating')

In [90]:
paramgrid=ParamGridBuilder().addGrid(als.regParam,[0.3,0.01,0.15]).addGrid(als.rank,range(2,8)).build()

In [91]:
evaluator=RegressionEvaluator(metricName='rmse',predictionCol='prediction',labelCol='rating')

In [92]:
crossval=CrossValidator(estimator=als,estimatorParamMaps=paramgrid,evaluator=evaluator,numFolds=5)

In [93]:
crossmodel=crossval.fit(df_data_tarin)

In [94]:
best_model=crossmodel.bestModel

In [100]:
predictions=best_model.transform(df_data_test)

In [96]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|      1|   5.0| 3.5930758|
|     1|      7|   4.0| 3.6004226|
|     1|     10|   3.0| 3.6383452|
|     1|     16|   5.0|  3.093947|
|     1|     17|   3.0| 2.8954697|
|     1|     34|   2.0|  2.085077|
|     1|     39|   4.0| 3.0860734|
|     1|     41|   2.0| 2.7528162|
|     1|     45|   5.0|  3.847292|
|     1|     51|   4.0| 3.1340594|
|     1|     56|   4.0| 3.8344424|
|     1|     57|   5.0| 3.6827676|
|     1|     59|   5.0| 3.8919723|
|     1|     66|   4.0|   3.10332|
|     1|     68|   4.0| 3.1840281|
|     1|     75|   4.0| 2.9401517|
|     1|     76|   4.0| 3.2980647|
|     1|     81|   5.0| 3.4151716|
|     1|     82|   5.0|  3.434549|
|     1|     87|   5.0| 3.6772847|
+------+-------+------+----------+
only showing top 20 rows



In [104]:
rmse=evaluator.evaluate(predictions.filter(col('prediction')!='nan'))

In [105]:
print(rmse)

0.9753243913726617


In [99]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|      1|   5.0| 3.5930758|
|     1|      7|   4.0| 3.6004226|
|     1|     10|   3.0| 3.6383452|
|     1|     16|   5.0|  3.093947|
|     1|     17|   3.0| 2.8954697|
|     1|     34|   2.0|  2.085077|
|     1|     39|   4.0| 3.0860734|
|     1|     41|   2.0| 2.7528162|
|     1|     45|   5.0|  3.847292|
|     1|     51|   4.0| 3.1340594|
|     1|     56|   4.0| 3.8344424|
|     1|     57|   5.0| 3.6827676|
|     1|     59|   5.0| 3.8919723|
|     1|     66|   4.0|   3.10332|
|     1|     68|   4.0| 3.1840281|
|     1|     75|   4.0| 2.9401517|
|     1|     76|   4.0| 3.2980647|
|     1|     81|   5.0| 3.4151716|
|     1|     82|   5.0|  3.434549|
|     1|     87|   5.0| 3.6772847|
+------+-------+------+----------+
only showing top 20 rows



In [13]:
import os
df_data_tarin,df_data_test=df_data.randomSplit([0.8,0.2])
als=ALS(userCol='userId',itemCol='movieId',ratingCol='rating')
paramgrid=ParamGridBuilder().addGrid(als.regParam,[0.3,0.01,0.15]).addGrid(als.rank,range(2,8)).build()
evaluator=RegressionEvaluator(metricName='rmse',predictionCol='prediction',labelCol='rating')
crossval=CrossValidator(estimator=als,estimatorParamMaps=paramgrid,evaluator=evaluator,numFolds=5)
crossmodel=crossval.fit(df_data_tarin)
best_model=crossmodel.bestModel
predictions=best_model.transform(df_data_test)
rmse=evaluator.evaluate(predictions.filter(col('prediction')!='nan'))
#model_path = f'file:///home/ali/Desktop/Movie Recommendation System/model/als_model_{datetime.date.today().strftime("%Y-%m-%d")}_{rmse}'
#os.makedirs(os.path.dirname(model_path), exist_ok=True)
#best_model.save(model_path)
    

In [109]:
spark.sparkContext._jsc.hadoopConfiguration().get("fs.defaultFS")

'hdfs://localhost:9000'

In [16]:
recomendation=best_model.recommendForAllUsers(5)


In [17]:
recomendation.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{1629, 4.907116}...|
|     2|[{1629, 4.870393}...|
|     3|[{1629, 3.8808732...|
|     4|[{1629, 5.894905}...|
|     5|[{1629, 4.076516}...|
|     6|[{1629, 4.5347886...|
|     7|[{1629, 5.192302}...|
|     8|[{1629, 5.0268207...|
|     9|[{1629, 5.2419786...|
|    10|[{1629, 5.2083077...|
|    11|[{1629, 4.627579}...|
|    12|[{1599, 5.321684}...|
|    13|[{1629, 4.313451}...|
|    14|[{1629, 4.980708}...|
|    15|[{1629, 4.0771594...|
|    16|[{1629, 5.4262524...|
|    17|[{1629, 4.1206236...|
|    18|[{1629, 4.834357}...|
|    19|[{1599, 4.1856008...|
|    20|[{1500, 4.004752}...|
+------+--------------------+
only showing top 20 rows



In [18]:
user_df = spark.createDataFrame([(1,)], ["userId"])

In [19]:
user_df.show()

+------+
|userId|
+------+
|     1|
+------+



In [ ]:
rec_user=best_model.recommendForUserSubset(user_df,5)

In [49]:
rec_user.show(truncate=False)

+------+--------------------------------------------------------------------------------------------+
|userId|recommendations                                                                             |
+------+--------------------------------------------------------------------------------------------+
|1     |[{1629, 4.907116}, {1599, 4.832293}, {1642, 4.791252}, {1467, 4.7408233}, {1398, 4.7407575}]|
+------+--------------------------------------------------------------------------------------------+



In [50]:
from pyspark.sql.functions import regexp_extract


In [51]:
rec_user=rec_user.selectExpr('*',"explode(recommendations) as rec").drop('recommendations')

In [59]:
rec_user.show()

+------+-----------------+-------+---------+
|userId|              rec|movieID|   rating|
+------+-----------------+-------+---------+
|     1| {1629, 4.907116}|   1629| 4.907116|
|     1| {1599, 4.832293}|   1599| 4.832293|
|     1| {1642, 4.791252}|   1642| 4.791252|
|     1|{1467, 4.7408233}|   1467|4.7408233|
|     1|{1398, 4.7407575}|   1398|4.7407575|
+------+-----------------+-------+---------+



In [58]:
rec_user.printSchema()



root
 |-- userId: integer (nullable = false)
 |-- rec: struct (nullable = true)
 |    |-- movieId: integer (nullable = true)
 |    |-- rating: float (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: float (nullable = true)



In [60]:
rec_user = rec_user.withColumn("movieID", col('rec.movieId')).withColumn("rating", col('rec.rating')).drop('rec')


In [61]:
rec_user.show()

+------+-------+---------+
|userId|movieID|   rating|
+------+-------+---------+
|     1|   1629| 4.907116|
|     1|   1599| 4.832293|
|     1|   1642| 4.791252|
|     1|   1467|4.7408233|
|     1|   1398|4.7407575|
+------+-------+---------+



In [65]:
rec_user.collect()

[Row(userId=1, movieID=1629, rating=4.907115936279297),
 Row(userId=1, movieID=1599, rating=4.8322930335998535),
 Row(userId=1, movieID=1642, rating=4.791252136230469),
 Row(userId=1, movieID=1467, rating=4.740823268890381),
 Row(userId=1, movieID=1398, rating=4.740757465362549)]

In [66]:
rec_es=[row.asDict() for row in rec_user.collect()]

In [67]:
rec_es

[{'userId': 1, 'movieID': 1629, 'rating': 4.907115936279297},
 {'userId': 1, 'movieID': 1599, 'rating': 4.8322930335998535},
 {'userId': 1, 'movieID': 1642, 'rating': 4.791252136230469},
 {'userId': 1, 'movieID': 1467, 'rating': 4.740823268890381},
 {'userId': 1, 'movieID': 1398, 'rating': 4.740757465362549}]

In [72]:

path = "file:///model/als_model_2025-03-13_0.9794150959395667"


In [84]:
import os
path = "/home/ali/Desktop/Movie Recommendation System/model/als_model_2025-03-13_0.9794150959395667"
print("Contents:", os.listdir(path[7:]) if os.path.exists(path[7:]) else "Nothing")

Contents: Nothing


In [76]:
path = "file:///home/ali/Desktop/Movie Recommendation System/model/als_model_2025-03-13_0.9794150959395667"
best_model = ALS.load(path)

Py4JJavaError: An error occurred while calling o17298.load.
: java.lang.NoSuchMethodException: org.apache.spark.ml.recommendation.ALSModel.<init>(java.lang.String)
	at java.lang.Class.getConstructor0(Class.java:3082)
	at java.lang.Class.getConstructor(Class.java:1825)
	at org.apache.spark.ml.util.DefaultParamsReader.load(ReadWrite.scala:468)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientSer

In [77]:
path = "/home/ali/Desktop/Movie Recommendation System/model/als_model_2025-03-13_0.9794150959395667"
best_model = ALS.load(path)

Py4JJavaError: An error occurred while calling o17300.load.
: org.apache.hadoop.ipc.RpcException: RPC response has invalid length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1933)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


In [78]:
from pyspark.ml.recommendation import ALS
path = "file:///home/ali/Desktop/Movie Recommendation System/model/als_model_2025-03-13_0.9794150959395667"
try:
    best_model = ALS.load(path)
    print("Model loaded successfully")
except Exception as e:
    print(f"Failed: {e}")

Failed: An error occurred while calling o17302.load.
: java.lang.NoSuchMethodException: org.apache.spark.ml.recommendation.ALSModel.<init>(java.lang.String)
	at java.lang.Class.getConstructor0(Class.java:3082)
	at java.lang.Class.getConstructor(Class.java:1825)
	at org.apache.spark.ml.util.DefaultParamsReader.load(ReadWrite.scala:468)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConn

In [79]:
from pyspark.ml.recommendation import ALS
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(df_data)
model.save("file:///home/ali/Desktop/Movie Recommendation System/model/als_model_2025-03-13_0.9794150959395667")

25/03/13 14:52:33 WARN MemoryManager: Total allocation exceeds 95.00% (952,316,711 bytes) of heap memory
Scaling row group sizes to 88.69% for 8 writers


In [80]:
from pyspark.ml.recommendation import ALS
path = "file:///home/ali/Desktop/Movie Recommendation System/model/als_model_2025-03-13_0.9794150959395667"
best_model = ALS.load(path)
print("Loaded successfully")

Py4JJavaError: An error occurred while calling o17386.load.
: java.lang.NoSuchMethodException: org.apache.spark.ml.recommendation.ALSModel.<init>(java.lang.String)
	at java.lang.Class.getConstructor0(Class.java:3082)
	at java.lang.Class.getConstructor(Class.java:1825)
	at org.apache.spark.ml.util.DefaultParamsReader.load(ReadWrite.scala:468)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientSer

In [81]:
print(spark.version)

3.5.1


In [85]:
import pyspark
print(pyspark.__version__)

3.5.1


In [87]:
from pyspark.ml.recommendation import ALSModel
model = ALSModel.load("file:///home/ali/Desktop/Movie Recommendation System/model/als_model_2025-03-13_0.9794150959395667")

In [88]:
user_df = spark.createDataFrame([(1,)], ["userId"])
rec_user=model.recommendForUserSubset(user_df,5)

In [89]:
rec_user.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{1449, 5.0420194...|
+------+--------------------+

